In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
from qgsw.utils.sorting import sort_files
from mpl_toolkits.axes_grid1 import make_axes_locatable
from qgsw.run_summary import RunSummary
from matplotlib.axes import Axes

In [ ]:
ROOT = Path(os.path.abspath('')).parent

In [ ]:
folder = ROOT.joinpath("output/g5k/two_layers_half_barotropic_100km")
run = RunSummary.from_file(folder.joinpath("_summary.toml"))
model = run.configuration.model
steps, files = sort_files(folder.glob(f"{model.prefix}*.npz"), prefix=model.prefix, suffix=".npz")

In [ ]:
%matplotlib tk
plt.ion()

fig = plt.figure(figsize=(12,18))

shape = (3,2)

top_axes: Axes = plt.subplot2grid(fig=fig, shape=shape, loc=(0,0))
top_cbar_axes: Axes = make_axes_locatable(top_axes).append_axes("right", size="7%", pad="2%")
bottom_axes: Axes = plt.subplot2grid(fig=fig, shape=shape, loc=(0,1))
bottom_cbar_axes: Axes = make_axes_locatable(bottom_axes).append_axes("right", size="7%", pad="2%")
ratio_axes:Axes = plt.subplot2grid(fig=fig, shape=shape, loc=(1,0), rowspan=1, colspan=1)
ratio_cbar_axes: Axes = make_axes_locatable(ratio_axes).append_axes("right", size="7%", pad="2%")
ratio_axes_:Axes = plt.subplot2grid(fig=fig, shape=shape, loc=(1,1), rowspan=1, colspan=1)
ratio_cbar_axes_: Axes = make_axes_locatable(ratio_axes_).append_axes("right", size="7%", pad="2%")
rmses: Axes = plt.subplot2grid(fig=fig, shape=shape, loc=(2,0), rowspan=1, colspan=2) 

rmse = []
rmse_ = []

end = 100
for i, file in enumerate(files[:end]):
    step = steps[i]
    fig.suptitle(f"Vorticity Ratio Estimation\nTime: {step*run.dt} sec / {steps[end]*run.dt} sec")
    omega = np.load(file)["omega"][...,10:-10, 10:-10]

    top_axes.cla()
    top_axes.set_title("Top Layer Potential Vorticity")
    top_cbar_axes.cla()
    bottom_axes.cla()
    bottom_axes.set_title("Bottom Layer Potential Vorticity")
    bottom_cbar_axes.cla()
    ratio_axes.cla()
    ratio_axes.set_title("Bottom / Top Potential Vorticity Ratio")
    ratio_cbar_axes.cla()
    ratio_axes_.cla()
    ratio_axes_.set_title("Bottom / Top Potential Vorticity Ratio")
    ratio_cbar_axes_.cla()
    rmses.cla()
    rmses.set_title("Ratio Comparison")

    top: np.ndarray = omega[0,0,...]
    bottom: np.ndarray = omega[0,1,...]

    is_not_zero = np.abs(top) > 1e-1

    top_cbar = top_axes.imshow(top, vmin=-np.max(np.abs(top)), vmax=np.max(np.abs(top)), cmap=plt.colormaps["bwr"])
    fig.colorbar(top_cbar, cax=top_cbar_axes)
    bottom_cbar = bottom_axes.imshow(bottom, vmin=-np.max(np.abs(bottom)), vmax=np.max(np.abs(bottom)), cmap=plt.colormaps["bwr"])
    fig.colorbar(bottom_cbar, cax=bottom_cbar_axes)

    ratio_map = np.zeros(top.shape)
    ratio = bottom[is_not_zero]/ (top[is_not_zero] + bottom[is_not_zero])
    alpha = ratio / (1 - ratio)
    ratio_map[is_not_zero] = alpha
    ratio_cbar = ratio_axes.imshow(ratio_map, vmin=-1, vmax=1, cmap=plt.colormaps["bwr"])
    fig.colorbar(ratio_cbar, cax=ratio_cbar_axes)

    ratio_map_ = np.zeros(top.shape)
    ratio_ = bottom[is_not_zero] / top[is_not_zero]
    alpha_ = ratio_
    ratio_map_[is_not_zero] = alpha_
    ratio_cbar_ = ratio_axes_.imshow(ratio_map_, vmin=-1, vmax=1, cmap=plt.colormaps["bwr"])
    fig.colorbar(ratio_cbar_, cax=ratio_cbar_axes_)

    rmse.append(np.mean(alpha))
    rmse_.append(np.mean(alpha_))

    rmses.plot(steps[:i+1], rmse, c='blue', label = "Modified Ratio")
    rmses.plot(steps[:i+1], rmse_, c='orange', label = "Standard Ratio")
    rmses.set_xlim(steps[0], steps[end])
    rmses.axvline(x=50000, alpha=0.5, c='red')
    rmses.legend()

    plt.pause(0.01)

plt.ioff()
plt.close()